In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
! pip install openai
! pip install tiktoken

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/cb/bb/c51fa42d85f431b3b3ec4c35a13a8bb99cafc0671918139a48767421d354/pinecone_client-3.2.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.9 kB ? eta -:--:--
   --- ----------------------------------- 20.5/215.9 kB 217.9 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/215.9 kB 245.8 kB/s eta 0:00:01
   --------- ----------------------------- 51.2/215.9 kB 238.1 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/215.9 kB 327.3 kB/s eta 0:00:01
   ----------------------- -------------- 133.1/215.9 kB 462.0 kB/s eta 0:00:01
   -------------------------------- ----- 184.3/215.9 kB 556.2 kB/s eta 0:00:01
   -------------------------------------- 215.9/215.9 kB 571.3 kB/s eta 0:00:00


In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [2]:
!mkdir pdfs

In [2]:
loader=PyPDFDirectoryLoader("pdfs")

In [3]:
data=loader.load()

In [4]:
data[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20) #Basically here we jsut dividing our data into chunks.

In [6]:
text_chunks = text_splitter.split_documents(data)

In [7]:
text_chunks

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'pdfs\\Transformer.pdf', 'page': 0}),
 Document(page_content='aidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple networ

In [8]:
text_chunks[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'pdfs\\Transformer.pdf', 'page': 0})

In [9]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [10]:
print(text_chunks[1].page_content)

aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,


In [11]:
print(text_chunks[2].page_content)

based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,


In [12]:
len(text_chunks)

91

In [13]:
print(text_chunks[90].page_content)

just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention
at layer 5 of 6. The heads clearly learned to perform different tasks.
15


In [14]:
import os
os.environ["OPENAI_API_KEY"]=""

In [15]:
embedding=OpenAIEmbeddings()

c:\Users\DELL\anaconda3\envs\python13\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:
embedding.embed_query("How are you")

[-0.016171867947776714,
 -0.00278568344715241,
 -0.004810338093806201,
 -0.03635182392278897,
 -0.023807526109539037,
 0.016615801453539348,
 -0.028995208614565993,
 -0.02004043268559754,
 -0.023388959966584976,
 -0.005130604541723888,
 0.02530421727905022,
 0.005593563569592951,
 -0.010521226880541953,
 0.003985889954963628,
 -0.01005826738701163,
 -0.01547425801996079,
 0.04375917209398405,
 -0.009259187076638892,
 0.008003489579099235,
 -0.014725912435205197,
 0.002363946616677909,
 0.009690436900997239,
 0.0006052559121646146,
 -0.002707995083643949,
 -0.016970948258149453,
 0.00015071943047532773,
 0.007121963942614852,
 -0.015677199716396934,
 0.004708867245588127,
 -0.02841175181938865,
 0.003307305549254133,
 -0.00549526434021784,
 -0.005790162959665694,
 0.0026350632170774107,
 0.007394665886774575,
 -0.021283446501732913,
 -0.00917674268184977,
 0.0030425307895567744,
 0.02185421868418345,
 0.005143288223128175,
 0.019837491920483917,
 -0.00011227160648902415,
 -0.00576479513

In [17]:
len(embedding.embed_query("How are you"))

1536

Dimension of OpenAI embedding is 1536.Demension is nothing it's just a feature.Simalarly Word2rec have 300 features.By using this feature they are representing a data.What should be the matrix for the simalarity search so there is a three option.

* Dot product
* Euclidean 
* Cosine 

In [18]:
len(embedding.embed_query("hi i am fine"))

1536

In [53]:
!pip install -U langchain-pinecone

  Obtaining dependency information for langchain-pinecone from https://files.pythonhosted.org/packages/7c/1f/df38fcf4604d826dcc949fb8f36074a326ea70870dddcf9123cefa6a9275/langchain_pinecone-0.1.0-py3-none-any.whl.metadata


In [19]:
from pinecone import Pinecone

c:\Users\DELL\anaconda3\envs\python13\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [27]:
#PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [28]:
#pinecone.init( api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
#)

#

In [35]:
pc = Pinecone(api_key='')

In [36]:
index_name = "testing" # put in the name of your pinecone index here

In [41]:
index = pc.Index(index_name)

## Create Embeddings for each of the Text Chunk

In [38]:
data=[t.page_content for t in text_chunks]

In [42]:
from langchain_pinecone import PineconeVectorStore

vectorstore_from_texts = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embedding
    )

PineconeConfigurationError: You haven't specified an Api-Key.

In [25]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

AttributeError: type object 'Pinecone' has no attribute 'from_texts'

In [36]:
docsearch

In [44]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7f1fa7300df0>)

In [37]:
query= "YOLOv7 outperforms which models"

In [38]:
docs = docsearch.similarity_search(query)

In [39]:
docs

[Document(page_content='YOLOv7-tiny 6.2 3.5 320 30.8% 47.3% 32.2% 10.0% 31.9% 52.2%\nimprovement -39% -49% - = = = -0.9 = +0.7\nYOLOR-E6 [81] 115.8M 683.2G 1280 55.7% 73.2% 60.7% 40.1% 60.4% 69.2%\nYOLOv7-E6 97.2M 515.2G 1280 55.9% 73.5% 61.1% 40.6% 60.3% 70.0%\nimprovement -19% -33% - +0.2 +0.3 +0.4 +0.5 -0.1 +0.8\nYOLOR-D6 [81] 151.7M 935.6G 1280 56.1% 73.9% 61.2% 42.4% 60.5% 69.9%\nYOLOv7-D6 154.7M 806.8G 1280 56.3% 73.8% 61.4% 41.3% 60.6% 70.1%\nYOLOv7-E6E 151.7M 843.2G 1280 56.8% 74.4% 62.1% 40.8% 62.1% 70.6%'),
 Document(page_content='YOLOv5-L6 (r6.1) [23] 76.8M 445.6G 1280 63 - / 53.7% - -\nYOLOX-X [21] 99.1M 281.9G 640 58 51.5% / 51.1% - -\nYOLOv7-E6 97.2M 515.2G 1280 56 56.0% /55.9% 73.5% 61.2%\nYOLOR-E6 [81] 115.8M 683.2G 1280 45 55.8% / 55.7% 73.4% 61.1%\nPPYOLOE-X [85] 98.4M 206.6G 640 45 52.2% / 51.9% 69.9% 56.5%\nYOLOv7-D6 154.7M 806.8G 1280 44 56.6% /56.3% 74.0% 61.8%\nYOLOv5-X6 (r6.1) [23] 140.7M 839.2G 1280 38 - / 55.0% - -\nYOLOv7-E6E 151.7M 843.2G 1280 36 56.8% /56.8

In [40]:
llm = OpenAI()

In [41]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [42]:
query = "YOLOv7 outperforms which models"

In [43]:
qa.run(query)

' YOLOv7 outperforms YOLOv5-L6, YOLOX-X, YOLOR-E6, PPYOLOE-X, YOLOv5-X6, and YOLOR-CSP.'

In [45]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt:  what is a yolo?


Answer:  YOLO (You Only Look Once) is a type of object detector, specifically a deep learning algorithm used for object detection in images and videos. It was developed by Chien-Yao Wang, Alexey Bochkovskiy, and Hong-Yuan Mark Liao from the Institute of Information Science at Academia Sinica in Taiwan. YOLOv7 is the latest version of the algorithm, which has set a new state-of-the-art for real-time object detectors in terms of speed and accuracy.


Input Prompt:  who is invented the yolo?


Answer:  Joseph Redmon and Ali Farhadi


Input Prompt:  what was the accuracy of the yolov7?


Answer:  The accuracy of the YOLOv7 was 56.8% AP test-dev / 56.8% AP min-val.


Input Prompt:  exit


Exiting


SystemExit: 

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
